In [50]:
import re
import subprocess
import os
import json

In [51]:
_text = subprocess.run(['cyber', '-h'], capture_output=True)
_text

CompletedProcess(args=['cyber', '-h'], returncode=0, stdout=b'Bostrom Bootloader Hub\n\nUsage:\n  cyber [command]\n\nAvailable Commands:\n  add-genesis-account Add a genesis account to genesis.json\n  collect-gentxs      Collect genesis txs and output a genesis.json file\n  config              Create or query an application CLI configuration file\n  debug               Tool for helping with debugging your application\n  export              Export state to JSON\n  gentx               Generate a genesis tx carrying a self delegation\n  help                Help about any command\n  init                Initialize private validator, p2p, genesis, and application configuration files\n  keys                Manage your application\'s keys\n  migrate             Migrate genesis to a specified target version\n  query               Querying subcommands\n  rollback            rollback cosmos-sdk and tendermint state by one height\n  start               Run the full node\n  status              Quer

In [60]:
_text = subprocess.run(['cyber', '-h'], capture_output=True)
_lines = _text.stdout.decode("utf-8").split("\n")
_lines


['Bostrom Bootloader Hub',
 '',
 'Usage:',
 '  cyber [command]',
 '',
 'Available Commands:',
 '  add-genesis-account Add a genesis account to genesis.json',
 '  collect-gentxs      Collect genesis txs and output a genesis.json file',
 '  config              Create or query an application CLI configuration file',
 '  debug               Tool for helping with debugging your application',
 '  export              Export state to JSON',
 '  gentx               Generate a genesis tx carrying a self delegation',
 '  help                Help about any command',
 '  init                Initialize private validator, p2p, genesis, and application configuration files',
 "  keys                Manage your application's keys",
 '  migrate             Migrate genesis to a specified target version',
 '  query               Querying subcommands',
 '  rollback            rollback cosmos-sdk and tendermint state by one height',
 '  start               Run the full node',
 '  status              Query re

In [ ]:


_text = subprocess.run(['cyber', '-h'], capture_output=True)
_lines = _text.stdout.decode("utf-8").split("\n")
_lines


['Bostrom Bootloader Hub',
 '',
 'Usage:',
 '  cyber [command]',
 '',
 'Available Commands:',
 '  add-genesis-account Add a genesis account to genesis.json',
 '  collect-gentxs      Collect genesis txs and output a genesis.json file',
 '  config              Create or query an application CLI configuration file',
 '  debug               Tool for helping with debugging your application',
 '  export              Export state to JSON',
 '  gentx               Generate a genesis tx carrying a self delegation',
 '  help                Help about any command',
 '  init                Initialize private validator, p2p, genesis, and application configuration files',
 "  keys                Manage your application's keys",
 '  migrate             Migrate genesis to a specified target version',
 '  query               Querying subcommands',
 '  rollback            rollback cosmos-sdk and tendermint state by one height',
 '  start               Run the full node',
 '  status              Query re

In [55]:
_commands_dict['Available Commands']

['  add-genesis-account Add a genesis account to genesis.json',
 '  collect-gentxs      Collect genesis txs and output a genesis.json file',
 '  config              Create or query an application CLI configuration file',
 '  debug               Tool for helping with debugging your application',
 '  export              Export state to JSON',
 '  gentx               Generate a genesis tx carrying a self delegation',
 '  help                Help about any command',
 '  init                Initialize private validator, p2p, genesis, and application configuration files',
 "  keys                Manage your application's keys",
 '  migrate             Migrate genesis to a specified target version',
 '  query               Querying subcommands',
 '  rollback            rollback cosmos-sdk and tendermint state by one height',
 '  start               Run the full node',
 '  status              Query remote node for status',
 '  tendermint          Tendermint subcommands',
 '  testnet           

In [59]:
_section = 'Available Commands'
_command_desc = []

for l in _commands_dict[_section]:
    if _match:= re.search(r'\s+(.*?)\s+(.*)', l):
        # print(_match.group(1), ";", _match.group(2))
        _command_desc.append((_match.group(1), _match.group(2),))

_command_desc        

[('add-genesis-account', 'Add a genesis account to genesis.json'),
 ('collect-gentxs', 'Collect genesis txs and output a genesis.json file'),
 ('config', 'Create or query an application CLI configuration file'),
 ('debug', 'Tool for helping with debugging your application'),
 ('export', 'Export state to JSON'),
 ('gentx', 'Generate a genesis tx carrying a self delegation'),
 ('help', 'Help about any command'),
 ('init',
  'Initialize private validator, p2p, genesis, and application configuration files'),
 ('keys', "Manage your application's keys"),
 ('migrate', 'Migrate genesis to a specified target version'),
 ('query', 'Querying subcommands'),
 ('rollback', 'rollback cosmos-sdk and tendermint state by one height'),
 ('start', 'Run the full node'),
 ('status', 'Query remote node for status'),
 ('tendermint', 'Tendermint subcommands'),
 ('testnet', 'Initialize files for a simapp testnet'),
 ('tx', 'Transactions subcommands'),
 ('validate-genesis',
  'validates the genesis file at the d

In [4]:
path = "/Users/user/apps-files/github/cyber_completitions/json-files/cyber-query distribution.json"

with open(path, "r") as f:
    content = f.read()

In [15]:
import pandas as pd

pd.DataFrame.from_records(json.loads(content))

,_command_root,_command,_description
0,query distribution,commission,Query distribution validator commission
1,query distribution,community-pool,Query the amount of coins in the community pool
2,query distribution,params,Query distribution params
3,query distribution,rewards,Query all distribution delegator rewards or re...
4,query distribution,slashes,Query distribution validator slashes
5,query distribution,validator-outstanding-rewards,Query distribution outstanding (un-withdrawn) ...


In [18]:
_list = []

for _file in os.scandir("/Users/user/apps-files/github/cyber_completitions/json-files/"):
    with open(_file, "r") as f:
        content = f.read()
        _list.append(json.loads(content))

In [26]:
[i for i in _list if i != []]

[[{'_command_root': '',
   '_command': 'add-genesis-account',
   '_description': 'Add a genesis account to genesis.json'},
  {'_command_root': '',
   '_command': 'collect-gentxs',
   '_description': 'Collect genesis txs and output a genesis.json file'},
  {'_command_root': '',
   '_command': 'config',
   '_description': 'Create or query an application CLI configuration file'},
  {'_command_root': '',
   '_command': 'debug',
   '_description': 'Tool for helping with debugging your application'},
  {'_command_root': '',
   '_command': 'export',
   '_description': 'Export state to JSON'},
  {'_command_root': '',
   '_command': 'gentx',
   '_description': 'Generate a genesis tx carrying a self delegation'},
  {'_command_root': '',
   '_command': 'help',
   '_description': 'Help about any command'},
  {'_command_root': '',
   '_command': 'init',
   '_description': 'Initialize private validator, p2p, genesis, and application configuration files'},
  {'_command_root': '',
   '_command': 'keys

In [25]:
df = pd.concat([pd.DataFrame.from_records(i) for i in _list if i != []])
df

,_command_root,_command,_description
0,,add-genesis-account,Add a genesis account to genesis.json
1,,collect-gentxs,Collect genesis txs and output a genesis.json ...
2,,config,Create or query an application CLI configurati...
3,,debug,Tool for helping with debugging your application
4,,export,Export state to JSON
...,...,...,...
5,query gov,proposals,Query proposals with optional filters
6,query gov,proposer,Query the proposer of a governance proposal
7,query gov,tally,Get the tally of a proposal vote
8,query gov,vote,Query details of a single vote


In [30]:
df['com1'] = "cyber " + df['_command_root'] + " " + df['_command']
df

,_command_root,_command,_description,com1
0,,add-genesis-account,Add a genesis account to genesis.json,cyber add-genesis-account
1,,collect-gentxs,Collect genesis txs and output a genesis.json ...,cyber collect-gentxs
2,,config,Create or query an application CLI configurati...,cyber config
3,,debug,Tool for helping with debugging your application,cyber debug
4,,export,Export state to JSON,cyber export
...,...,...,...,...
5,query gov,proposals,Query proposals with optional filters,cyber query gov proposals
6,query gov,proposer,Query the proposer of a governance proposal,cyber query gov proposer
7,query gov,tally,Get the tally of a proposal vote,cyber query gov tally
8,query gov,vote,Query details of a single vote,cyber query gov vote


In [40]:
_l2 = df.loc[:, ['_description', 'com1']].to_dict('records')
_l2

[{'_description': 'Add a genesis account to genesis.json',
  'com1': 'cyber  add-genesis-account'},
 {'_description': 'Collect genesis txs and output a genesis.json file',
  'com1': 'cyber  collect-gentxs'},
 {'_description': 'Create or query an application CLI configuration file',
  'com1': 'cyber  config'},
 {'_description': 'Tool for helping with debugging your application',
  'com1': 'cyber  debug'},
 {'_description': 'Export state to JSON', 'com1': 'cyber  export'},
 {'_description': 'Generate a genesis tx carrying a self delegation',
  'com1': 'cyber  gentx'},
 {'_description': 'Help about any command', 'com1': 'cyber  help'},
 {'_description': 'Initialize private validator, p2p, genesis, and application configuration files',
  'com1': 'cyber  init'},
 {'_description': "Manage your application's keys", 'com1': 'cyber  keys'},
 {'_description': 'Migrate genesis to a specified target version',
  'com1': 'cyber  migrate'},
 {'_description': 'Querying subcommands', 'com1': 'cyber  qu

In [44]:
for l in _l2:
    print(f'# {l["_description"]}\nextern "{l["com1"]}" [] \n')

# Add a genesis account to genesis.json
extern "cyber  add-genesis-account" [] 

# Collect genesis txs and output a genesis.json file
extern "cyber  collect-gentxs" [] 

# Create or query an application CLI configuration file
extern "cyber  config" [] 

# Tool for helping with debugging your application
extern "cyber  debug" [] 

# Export state to JSON
extern "cyber  export" [] 

# Generate a genesis tx carrying a self delegation
extern "cyber  gentx" [] 

# Help about any command
extern "cyber  help" [] 

# Initialize private validator, p2p, genesis, and application configuration files
extern "cyber  init" [] 

# Manage your application's keys
extern "cyber  keys" [] 

# Migrate genesis to a specified target version
extern "cyber  migrate" [] 

# Querying subcommands
extern "cyber  query" [] 

# rollback cosmos-sdk and tendermint state by one height
extern "cyber  rollback" [] 

# Run the full node
extern "cyber  start" [] 

# Query remote node for status
extern "cyber  status" [] 

#

In [10]:
for i in json.loads(content):
    print("\t", i['_command'], "\t\t#", i['_description'])

	 commission 		# Query distribution validator commission
	 community-pool 		# Query the amount of coins in the community pool
	 params 		# Query distribution params
	 rewards 		# Query all distribution delegator rewards or rewards from a particular validator
	 slashes 		# Query distribution validator slashes
	 validator-outstanding-rewards 		# Query distribution outstanding (un-withdrawn) rewards for a validator and all their delegations
